# Purpose:

2015-03-18 (Wednesday)

Query the database.

# Table of Searches:

[How-many-positives-did-we-find-after-Gisella-left-in-July?](#How-many-positives-did-we-find-after-Gisella-left-in-July?)

[Get-next-set-of-fly-IDs-for-Robert's-MicroSat-work](#Get-next-set-of-fly-IDs-for-Robert's-MicroSat-work.)

## Imports:

In [56]:
# imports
import datetime as dt

import csv
import itertools
import os
import re

import pandas as pd
pd.options.display.max_rows = 1000
import numpy as np
import datetime as dt

import scipy

import munch


In [2]:
# for plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import ggplot as gp

import ggplot as g


## File paths:

In [3]:
# define paths to files
spring_summer = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/2014_spring_summer_from_rob.xlsx"
fall =          "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/2014_fall_for_pandas.xlsx"
december = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/DEC_2014_survey_for_pandas.xlsx"

h5_out_path = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/hdf5/2014_collection_records.h5"
json_out_path = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/hdf5/2014_collection_records.json"
pickle_out_path = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/hdf5/2014_collection_records.pkl"

In [4]:
village_id_map_path = "/home/gus/Dropbox/uganda_data/data_repos/field_data/locations/names/uganda_village_id_map.csv"

----
# Helper functions

In [5]:
def date_is_between(test_date, start, end):
    
    try:
        return start <= test_date <= end
    except TypeError as exc:
        if "can't compare datetime" in exc.message:
            return False
        else:
            raise

In [6]:
def get_village_id_map(village_id_map_path):
    """
    Generates and returns a `dict` mapping the long-form village names to the letter codes.
    Letter codes map back to themselves to ensure a one way mapping.  
    Enforces both be all UPPERcase to allow case insensitivity as long as
    the map is used like: `map[text.upper()]`.
    :return: `dict`
    """

    village_id_map = {}

    with open(village_id_map_path, 'rb') as csv_file:
        village_ids = csv.reader(csv_file, delimiter=',')
        for pair in village_ids:
            village_id_map[unicode(pair[0].upper())] = unicode(pair[0].upper())
            village_id_map[unicode(pair[1].upper())] = unicode(pair[0].upper())

    return village_id_map

In [7]:
def load_xl_sheets(xl_path):
    dfs = munch.Munch()
    
    xls = pd.ExcelFile(xl_path)
    
    workbook_name = os.path.basename(xl_path)
    
    for sheet in xls.sheet_names:
        if sheet.upper().startswith("DISSECT"):
            worksheet_df = xls.parse(sheetname=sheet, 
                                header=0, 
                                skiprows=None, skip_footer=0, 
                                index_col=None, parse_cols=None, 
                                parse_dates=False, date_parser=None, 
                                na_values=['NA'], 
                                thousands=None, chunksize=None, 
                                convert_float=False, 
                                has_index_names=False, converters=None)
            
            worksheet_df['workbook'] = workbook_name
            worksheet_df['worksheet'] = sheet
            
            dfs[sheet] = worksheet_df 
            
    return dfs

In [8]:
def recode_villages(df):
    map_func = lambda x: village_id_map[x.upper()]
        
    new_codes = df.Village.apply(map_func)
    df.Village = new_codes    

In [9]:
def recode_dead(df):
    def recode_func(x):
        # this is treated as an unknown case
        if pd.isnull(x):
            return x

        x = unicode(x)

        # True means DEAD
        # False means LIVE or NOT-DEAD
        # None means unknown

        try:
            # deal with Live type cases
            if x.upper().startswith('L'):
                return False


            if x.startswith('0'):
                return False


            # deal with Dead type cases
            if x.upper().startswith('D'):
                return True


            if x.startswith('1'):
                return True


            # deal with unknown type cases
            if x.upper().startswith('UN'):
                return None
        except AttributeError:
            return x

        msg = "The value {x} was not expected and this function must be corrected to continue.".format(x=x)
        raise ValueError(msg)

    new_dead = df.Dead.apply(recode_func)
    df.Dead = new_dead

##########################################

def recode_teneral(df):
    def recode_func(x):

        # this is treated as an unknown case
        if pd.isnull(x):
            return x

        x = unicode(x)

        # True means teneral
        # False means NOT-teneral
        # None means unknown

        try:
            # deal with NOT-teneral type cases
            if x.upper().startswith('N'):
                return False

            if x.startswith('0'):
                return False

            # deal with Teneral type cases
            if x.upper().startswith('T'):
                return True
            
            if x.startswith('1'):
                return True


            # Deal with unknown type cases
            if x.upper().startswith('UN'):
                return x
        except AttributeError:
            return x

        msg = "The value {x} was not expected and this function must be corrected to continue.".format(x=x)
        raise ValueError(msg)
    
    
    new_teneral = df.Teneral.apply(recode_func)
    df.Teneral = new_teneral

##########################################

def recode_positives(df):
    def recode_func(x):
        # this is treated as an unknown case
        if pd.isnull(x):
            return x

        y = unicode(x)

        # deal with Unknown type cases
        if y.upper().startswith('UN'):
            return None

        if y.upper().startswith('DEAD'):
            return None


        # deal with Positive type cases
        if y.startswith('1'):
            return True


        if y.upper().startswith('TRUE'):
            return True

        if y.upper().startswith('P'):
            return True

        if y.upper().startswith('Y'):
            return True


        # deal with Negative type cases
        if y.upper().startswith('NO'):
            return False

        if y.upper().startswith('FALSE'):
            return False


        if y.startswith('0'):
            return False


        msg = "The value {x} was not expected and this function must be corrected to continue.".format(x=x)
        raise ValueError(msg)


    new_prob = df.prob.apply(recode_func)
    df.prob = new_prob
    
    new_midgut = df.midgut.apply(recode_func)
    df.midgut = new_midgut
    
    new_sal_gland = df.sal_gland.apply(recode_func)
    df.sal_gland = new_sal_gland

##########################################

def recode_species(df):

    recode_func = lambda x: ''.join(x.split('.')).capitalize()

    new_Species = df.Species.apply(recode_func)
    df.Species = new_Species

##########################################

def recode_sex(df):

    recode_func = lambda x: x.upper()

    new_Sex = df.Sex.apply(recode_func)
    df.Sex = new_Sex
    
##########################################

date_delim = re.compile('[\./-]')

def cast_unicode_as_date(x):
    if not isinstance(x, unicode):
        return x
    
    parts = date_delim.split(x)
    
    if len(parts) != 3:
        return x
    
    if len(parts[0]) != 4:
        return x
    
    return dt.datetime(int(parts[0]), int(parts[1]), int(parts[2]))

def recode_date(df):
    new_date = df.Date.apply(cast_unicode_as_date)
    df.Date = new_date

##########################################

fly_no_delim = re.compile('[\W\s]', re.UNICODE)

def split_number(x):
#     ipdb.set_trace()
    
    # to prevent unicode creating a string with a '.' AFTER
    # the numbert we are intersted in!
    try:
        if isinstance(x,float):
            return int(x)
    except ValueError as exc:
        if 'NAN' in exc.message.upper():
            return x
    
    x = unicode(x)
    parts = fly_no_delim.split(x)
    
    try:
        number = int(parts[-1])
        return number
    except ValueError:
        return x


def recode_fly_number(df):
    
    new_fly_number = df.Fly_Number.apply(split_number)
    df.Fly_Number = new_fly_number

In [10]:
def aggregate_column_from_df_list(df_list, col_name):
    agg_data = []
    for df in df_list:
        agg_data.extend(list(df[col_name]))
        
    
    return agg_data

### Functions that add new columns

In [11]:
def add_infection_state_col(df):
    df['infection_state'] = df[['prob','midgut','sal_gland']].any(skipna=True,axis=1)      

## Recode Fly_Number

In [12]:
# pdb
# import ipdb

In [13]:
dfp = pd.read_pickle(pickle_out_path)
dfp.reset_index(level=0, inplace=True)
dfp.drop(['index'], axis=1, inplace=True)
recode_fly_number(dfp)
dfp = dfp.query("Species in ['Gff']")
dfp

,Comment,Date,Dead,Fly_Number,Hunger_stage,Kept_in,Sex,Species,Teneral,Trap_No,Village,Wing_fray,infection_state,midgut,prob,sal_gland,workbook,worksheet
0,NaN,2014-07-22,False,1,2,EtOH,F,Gff,False,1,UWA,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Nwoya
1,NaN,2014-07-22,False,2,3,EtOH,F,Gff,False,1,UWA,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Nwoya
2,NaN,2014-07-22,False,3,3,EtOH,M,Gff,False,1,UWA,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Nwoya
3,NaN,2014-07-22,False,4,3,EtOH,M,Gff,False,1,UWA,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Nwoya
4,NaN,2014-07-22,False,5,3,EtOH,M,Gff,False,1,UWA,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Nwoya
5,NaN,2014-07-22,False,6,2,EtOH,M,Gff,False,1,UWA,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Nwoya
6,NaN,2014-07-22,False,7,3,EtOH,F,Gff,False,1,UWA,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Nwoya
7,NaN,2014-07-22,False,8,3,EtOH,F,Gff,False,1,UWA,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Nwoya
8,NaN,2014-07-22,False,9,3,EtOH,F,Gff,False,1,UWA,3,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Nwoya
9,NaN,2014-07-22,False,10,2,EtOH,F,Gff,False,1,UWA,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Nwoya


----
# Helpful constants

In [14]:
village_id_map = get_village_id_map(village_id_map_path)

In [15]:
# pdb

In [16]:
# # Example pivot_table
# tableXXXX = pd.pivot_table(df.query("XXXX == XXXX"),index=['XXXX'],
#                       values=['XXXX'],
#                       columns=['XXXX'],
#                       fill_value=0,
#                       aggfunc=[len])
# tableXXXX.head()

---
# Query the data

### How many positives did we find after Gisella left in July?

In [17]:
date_mask = dfp.Date.apply(lambda x: date_is_between(x,
                                                          dt.datetime(2014, 7, 11),
                                                          dt.datetime(2014, 8, 11)))
post_gisella_july = dfp[date_mask]

In [18]:
post_gisella_july_positive = post_gisella_july.query('infection_state == True')

In [19]:
post_gisella_july_negative = post_gisella_july.query('infection_state == False')

In [20]:
len(post_gisella_july_positive)

30

In [21]:
len(post_gisella_july_negative)

2287

In [22]:
# post_gisella_july_positive.to_csv('/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/seraps_stuff/positives_from_gus_gisella_time.tsv', sep='\t')

In [23]:
# post_gisella_july_negative.to_csv('/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/seraps_stuff/negatives_from_gus_gisella_time.tsv', sep='\t')

In [24]:
def get_random_rows(df, sample_size, exclude_by=None, exclude_values=None):
    
    assert (exclude_by is None) or isinstance(exclude_by,str)
    assert (exclude_values is None) or isinstance(exclude_values,list)
    
    try:
        if (exclude_by is None) and (exclude_values is None):
            post_exclude = df
        else:
            post_exclude = df[df[exclude_by].apply(lambda x: x not in exclude_values)]
            
            
        indexes = np.random.choice(post_exclude.index, sample_size, replace=False)
        return df.loc[indexes]
    except ValueError as exc:
        if "Cannot take a larger sample than population when 'replace=False'" in exc.message:
            print "Returning all {vil}:{sex} bc sample_size is larger than population.".format(vil=post_exclude.Village.iloc[0],
                                                                                               sex=post_exclude.Sex.iloc[0])
            return post_exclude

### Get next set of fly IDs for Robert's MicroSat work.

- NGO 15 each M/F
- CHU 15 each M/F
- TUM 15 each M/F

In [25]:
# get_random_rows(dfp, 10)

In [26]:
# ngo_m = get_random_rows(dfp.query('Sex == "M"').query('Village == "NGO"'), 15)
# ngo_f = get_random_rows(dfp.query('Sex == "F"').query('Village == "NGO"'), 15)

# chu_m = get_random_rows(dfp.query('Sex == "M"').query('Village == "CHU"'), 15)
# chu_f = get_random_rows(dfp.query('Sex == "F"').query('Village == "CHU"'), 15)

# tum_m = get_random_rows(dfp.query('Sex == "M"').query('Village == "TUM"'), 15)
# tum_f = get_random_rows(dfp.query('Sex == "F"').query('Village == "TUM"'), 15)

In [27]:
def write_samples(dir_path,df,sep_sex=False):
    
    if sep_sex:
        file_name = "{date}_{num}_{sex}_random_{vil}.xls".format(
            date=dt.datetime.today().isoformat().split('T')[0],
            num=len(df),
            sex=df.Sex.iloc[0],
            vil=df.Village.iloc[0])
    else:
        file_name = "{date}_{num}_MF_random_{vil}.xls".format(
            date=dt.datetime.today().isoformat().split('T')[0],
            num=len(df),
            vil=df.Village.iloc[0])
    
    out_path = "{dir}/{fname}".format(dir=os.path.dirname(dir_path),
                                      fname=file_name)
    
#     df.to_excel(out_path)
    df.to_csv(out_path,sep='\t',date_format='%Y-%m-%d')
    print out_path
        

In [28]:
# dir_path = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/"


# write_samples(dir_path, ngo_m)
# write_samples(dir_path, ngo_f)

# write_samples(dir_path, chu_m)
# write_samples(dir_path, chu_f)

# write_samples(dir_path, tum_m)
# write_samples(dir_path, tum_f)

print """
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-21_15_M_random_sampled_tsetse_from_NGO.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-21_15_F_random_sampled_tsetse_from_NGO.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-21_15_M_random_sampled_tsetse_from_CHU.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-21_15_F_random_sampled_tsetse_from_CHU.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-21_15_M_random_sampled_tsetse_from_TUM.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-21_15_F_random_sampled_tsetse_from_TUM.xls
"""


/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-21_15_M_random_sampled_tsetse_from_NGO.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-21_15_F_random_sampled_tsetse_from_NGO.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-21_15_M_random_sampled_tsetse_from_CHU.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-21_15_F_random_sampled_tsetse_from_CHU.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-21_15_M_random_sampled_tsetse_from_TUM.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-21_15_F_random_sampled_tsetse_from_TUM.xls



In [29]:
# chu_m.head()

### 2015-03-23: Get next set of fly IDs for Robert's MicroSat work to Kirstin/Alexis 

- GAN 15 each M/F
- LEA 15 each M/F
- OSG 15 each M/F
- GOR 15 each M/F  (NOPE)
- OGU 15 each M/F
- APU 15 each M/F

In [30]:
# GAN_m = get_random_rows(dfp.query('Sex == "M"').query('Village == "GAN"'), 15)
# GAN_f = get_random_rows(dfp.query('Sex == "F"').query('Village == "GAN"'), 15)

# # LEA_m = get_random_rows(dfp.query('Sex == "M"').query('Village == "LEA"'), 15)
# # LEA_f = get_random_rows(dfp.query('Sex == "F"').query('Village == "LEA"'), 15)

# OSG_m = get_random_rows(dfp.query('Sex == "M"').query('Village == "OSG"'), 15)
# OSG_f = get_random_rows(dfp.query('Sex == "F"').query('Village == "OSG"'), 15)

# GOR_m = get_random_rows(dfp.query('Sex == "M"').query('Village == "GOR"'), 15)
# GOR_f = get_random_rows(dfp.query('Sex == "F"').query('Village == "GOR"'), 15)

# # OGU_m = get_random_rows(dfp.query('Sex == "M"').query('Village == "OGU"'), 15)
# # OGU_f = get_random_rows(dfp.query('Sex == "F"').query('Village == "OGU"'), 15)

# APU_m = get_random_rows(dfp.query('Sex == "M"').query('Village == "APU"'), 15)
# APU_f = get_random_rows(dfp.query('Sex == "F"').query('Village == "APU"'), 15)



# GAN_mf = pd.concat([GAN_m, GAN_f])
# # LEA_mf = pd.concat([LEA_m, LEA_f])
# OSG_mf = pd.concat([OSG_m, OSG_f])
# # GOR_mf = pd.concat([GOR_m, GOR_f])
# # OGU_mf = pd.concat([OGU_m, OGU_f])
# APU_mf = pd.concat([APU_m, APU_f])

In [31]:
dir_path = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/"


# write_samples(dir_path, GAN_mf)
# # write_samples(dir_path, LEA_mf)
# write_samples(dir_path, OSG_mf)
# write_samples(dir_path, GOR_mf)
# # write_samples(dir_path, OGU_mf)
# write_samples(dir_path, APU_mf)

print """
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-23_30_MF_random_GAN.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-23_30_MF_random_OSG.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-23_30_MF_random_APU.xls
"""


/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-23_30_MF_random_GAN.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-23_30_MF_random_OSG.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-23_30_MF_random_APU.xls



### 2015-03-24: Get next set of fly IDs for Robert's MicroSat work to Kirstin/Alexis 

- OKS 15 each M/F

In [32]:
# len(dfp.query('Village == "OKS"').query('Dead == False').query('Teneral == False').query('infection_state == False').query('Sex == "F"'))

In [33]:
# OKS_select = dfp.query('Village == "OKS"').query('Dead == False').query('Teneral == False').query('infection_state == False')

In [34]:
# OKS_select.query("Fly_Number == 175")

In [35]:
# OKS_m = get_random_rows(OKS_select.query('Sex == "M"'),15)
# OKS_f = get_random_rows(OKS_select.query('Sex == "F"'),15)

# OKS_mf = pd.concat([OKS_m, OKS_f])

In [36]:
# write_samples(dir_path, OKS_mf)

### 2015-03-23: Get next set of fly IDs for Robert's MicroSat work to Kirstin/Alexis 

- GAN 15 each M/F
- LEA 15 each M/F
- OSG 15 each M/F
- GOR 15 each M/F  (NOPE)
- OGU 15 each M/F
- APU 15 each M/F

In [37]:
osg_missing = [122,
               138,
               91,
               170,
               110,
               107,
               169,
               168,
               146,
               118,
               93,
               149,
               151,
               152,
               114,
               139,
               134,
               113,
               140
              ]

gan_missing = [280,
               151,
               436,
               105,
               307,
               154,
               432,
               314,
               312,
               147,
               308,
               435,
               433,
               259,
               296,
               255,
               126,
               261,
               298
              ]

In [38]:
# GAN_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "GAN"'), 13, 
#                         exclude_by='Fly_Number', exclude_values=gan_missing)
# GAN_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "GAN"'), 13, 
#                         exclude_by='Fly_Number', exclude_values=gan_missing)

# LEA_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "LEA"'), 13, 
#                         exclude_by=None, exclude_values=None)
# LEA_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "LEA"'), 13, 
#                         exclude_by=None, exclude_values=None)

# OSG_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "OSG"'), 13, 
#                         exclude_by='Fly_Number', exclude_values=osg_missing)
# OSG_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "OSG"'), 13, 
#                         exclude_by='Fly_Number', exclude_values=osg_missing)

# OGU_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "OGU"'), 13, 
#                         exclude_by=None, exclude_values=None)
# OGU_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "OGU"'), 13, 
#                         exclude_by=None, exclude_values=None)

# APU_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "APU"'), 13, 
#                         exclude_by=None, exclude_values=None)
# APU_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "APU"'), 13,
#                         exclude_by=None, exclude_values=None)



# print "{vil}: {leng}".format(vil=OSG_mf.Village.iloc[0], leng=len(OSG_mf))
# print "{vil}: {leng}".format(vil=OGU_mf.Village.iloc[0], leng=len(OGU_mf))
# print "{vil}: {leng}".format(vil=APU_mf.Village.iloc[0], leng=len(APU_mf))

Output of above:

    Returning all GAN:M bc sample_size is larger than population.
    Returning all LEA:M bc sample_size is larger than population.
    Returning all LEA:F bc sample_size is larger than population.
    Returning all OSG:M bc sample_size is larger than population.
    Returning all OGU:M bc sample_size is larger than population.
    GAN: 25
    LEA: 16
    OSG: 25
    OGU: 20
    APU: 26

In [39]:
dir_path = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/"


# write_samples(dir_path, GAN_mf)
# write_samples(dir_path, LEA_mf)
# write_samples(dir_path, OSG_mf)
# write_samples(dir_path, OGU_mf)
# write_samples(dir_path, APU_mf)


Output of above:

    /home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-31_25_MF_random_GAN.xls
    /home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-31_16_MF_random_LEA.xls
    /home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-31_25_MF_random_OSG.xls
    /home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-31_20_MF_random_OGU.xls
    /home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-03-31_26_MF_random_APU.xls

### 2015-04-10: Get next set of fly IDs for Robert's MicroSat work to Kirstin/Alexis 

- AKA 13 each M/F
- OLE 13 each M/F


In [40]:
# AKA_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "AKA"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# AKA_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "AKA"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)

# OLE_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "OLE"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# OLE_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "OLE"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)

# AKA_mf = pd.concat([AKA_m, AKA_f])
# OLE_mf = pd.concat([OLE_m, OLE_f])



# print "{vil}: {leng}".format(vil=AKA_mf.Village.iloc[0], leng=len(AKA_mf))
# print "{vil}: {leng}".format(vil=OLE_mf.Village.iloc[0], leng=len(OLE_mf))

In [41]:
# drop_columns = ['Comment','Hunger_stage','Kept_in','Wing_fray','midgut','prob','sal_gland','workbook','worksheet']

# AKA_mf = AKA_mf.sort(columns='Fly_Number', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# AKA_mf = AKA_mf.sort(columns='Date', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# AKA_mf.drop(drop_columns, axis=1, inplace=True)

# OLE_mf = OLE_mf.sort(columns='Fly_Number', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# OLE_mf = OLE_mf.sort(columns='Date', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# OLE_mf.drop(drop_columns, axis=1, inplace=True)

In [42]:
# write_samples(dir_path, AKA_mf)
# write_samples(dir_path, OLE_mf)

Output of above:     

    /home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-04-10_26_MF_random_AKA.xls
    /home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-04-10_24_MF_random_OLE.xls

In [43]:
# AKA_mf

# 2015-04-17: Get rest of fly IDs for Robert's MicroSat work to Kirstin/Alexis 

```
- AIN 13 each M/F
- OLO 13 each M/F
- LAG 13 each M/F
- BOL 13 each M/F
- KIL 13 each M/F
- ~~TEO~13~each~M/F~~ [All but 1 were not G. f. fuscipes and the Gff was not dissected]
- OCA 13 each M/F
- ABO 13 each M/F
- ~~OCL~13~each~M/F~~  [ZERO indviduals meet filtering criteria]
```

In [50]:
# AIN_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "AIN"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# AIN_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "AIN"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# AIN_mf = pd.concat([AIN_m, AIN_f])
# print "{vil}: {leng}".format(vil=AIN_mf.Village.iloc[0], leng=len(AIN_mf))
# #
OLO_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "OLO"').query('Teneral == False'), 13, 
                        exclude_by=None, exclude_values=None)
OLO_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "OLO"').query('Teneral == False'), 13, 
                        exclude_by=None, exclude_values=None)
OLO_mf = pd.concat([OLO_m, OLO_f])
# print "{vil}: {leng}".format(vil=OLO_mf.Village.iloc[0], leng=len(OLO_mf))
# #
# LAG_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "LAG"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# LAG_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "LAG"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# LAG_mf = pd.concat([LAG_m, LAG_f])
# print "{vil}: {leng}".format(vil=LAG_mf.Village.iloc[0], leng=len(LAG_mf))
# #
# BOL_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "BOL"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# BOL_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "BOL"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# BOL_mf = pd.concat([BOL_m, BOL_f])
# print "{vil}: {leng}".format(vil=BOL_mf.Village.iloc[0], leng=len(BOL_mf))
# #
# KIL_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "KIL"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# KIL_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "KIL"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# KIL_mf = pd.concat([KIL_m, KIL_f])
# print "{vil}: {leng}".format(vil=KIL_mf.Village.iloc[0], leng=len(KIL_mf))

# ###### TEO_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "TEO"').query('Teneral == False'), 13, 
# ######                         exclude_by=None, exclude_values=None)
# ###### TEO_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "TEO"').query('Teneral == False'), 13, 
# ######                         exclude_by=None, exclude_values=None)
# ###### TEO_mf = pd.concat([TEO_m, TEO_f])
# ###### print "{vil}: {leng}".format(vil=TEO_mf.Village.iloc[0], leng=len(TEO_mf))
# #
# OCA_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "OCA"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# OCA_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "OCA"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# OCA_mf = pd.concat([OCA_m, OCA_f])
# print "{vil}: {leng}".format(vil=OCA_mf.Village.iloc[0], leng=len(OCA_mf))
# #
# ABO_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "ABO"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# ABO_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "ABO"').query('Teneral == False'), 13, 
#                         exclude_by=None, exclude_values=None)
# ABO_mf = pd.concat([ABO_m, ABO_f])
# print "{vil}: {leng}".format(vil=ABO_mf.Village.iloc[0], leng=len(ABO_mf))
# #
# ###### OCL_m = get_random_rows(dfp.query('Dead == False').query('Sex == "M"').query('Village == "OCL"').query('Teneral == False'), 13, 
# ######                         exclude_by=None, exclude_values=None)
# ###### OCL_f = get_random_rows(dfp.query('Dead == False').query('Sex == "F"').query('Village == "OCL"').query('Teneral == False'), 13, 
# ######                         exclude_by=None, exclude_values=None)
# ###### OCL_mf = pd.concat([OCL_m, OCL_f])
# ###### print "{vil}: {leng}".format(vil=OCL_mf.Village.iloc[0], leng=len(OCL_mf))

Output of above:

```
AIN: 26
OLO: 26
Returning all LAG:M bc sample_size is larger than population.
LAG: 17
Returning all BOL:M bc sample_size is larger than population.
BOL: 20
Returning all KIL:M bc sample_size is larger than population.
KIL: 17
OCA: 26
Returning all ABO:M bc sample_size is larger than population.
ABO: 2
```

In [51]:
drop_columns = ['Hunger_stage','Kept_in','Wing_fray','midgut','prob','sal_gland','workbook','worksheet']

# AIN_mf = AIN_mf.sort(columns='Fly_Number', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# AIN_mf = AIN_mf.sort(columns='Date', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# AIN_mf.drop(drop_columns, axis=1, inplace=True)
# #
OLO_mf = OLO_mf.sort(columns='Fly_Number', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
OLO_mf = OLO_mf.sort(columns='Date', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
OLO_mf.drop(drop_columns, axis=1, inplace=True)
# #
# LAG_mf = LAG_mf.sort(columns='Fly_Number', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# LAG_mf = LAG_mf.sort(columns='Date', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# LAG_mf.drop(drop_columns, axis=1, inplace=True)
# #
# BOL_mf = BOL_mf.sort(columns='Fly_Number', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# BOL_mf = BOL_mf.sort(columns='Date', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# BOL_mf.drop(drop_columns, axis=1, inplace=True)
# #
# KIL_mf = KIL_mf.sort(columns='Fly_Number', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# KIL_mf = KIL_mf.sort(columns='Date', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# KIL_mf.drop(drop_columns, axis=1, inplace=True)
# #
# # TEO_mf = TEO_mf.sort(columns='Fly_Number', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# # TEO_mf = TEO_mf.sort(columns='Date', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# # TEO_mf.drop(drop_columns, axis=1, inplace=True)
# #
# OCA_mf = OCA_mf.sort(columns='Fly_Number', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# OCA_mf = OCA_mf.sort(columns='Date', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# OCA_mf.drop(drop_columns, axis=1, inplace=True)
# #
# ABO_mf = ABO_mf.sort(columns='Fly_Number', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# ABO_mf = ABO_mf.sort(columns='Date', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# ABO_mf.drop(drop_columns, axis=1, inplace=True)
# #
# # OCL_mf = OCL_mf.sort(columns='Fly_Number', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# # OCL_mf = OCL_mf.sort(columns='Date', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# # OCL_mf.drop(drop_columns, axis=1, inplace=True)
# #


In [52]:
# write_samples(dir_path, AIN_mf)
# write_samples(dir_path, OLO_mf)
# write_samples(dir_path, LAG_mf)
# write_samples(dir_path, BOL_mf)
# write_samples(dir_path, KIL_mf)
# # write_samples(dir_path, TEO_mf)
# write_samples(dir_path, OCA_mf)
# write_samples(dir_path, ABO_mf)
# # write_samples(dir_path, OCL_mf)

Output from above:

```

/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-04-17_26_MF_random_AIN.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-04-17_26_MF_random_OLO.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-04-17_17_MF_random_LAG.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-04-17_20_MF_random_BOL.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-04-17_17_MF_random_KIL.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-04-17_26_MF_random_OCA.xls
/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/samples_used/2015-04-17_2_MF_random_ABO.xls
```

In [48]:
OCA_mf = OCA_mf.sort(columns='Fly_Number', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
OCA_mf = OCA_mf.sort(columns='Date', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
OCA_mf

NameError: name 'OCA_mf' is not defined

In [53]:
OLO_mf = OLO_mf.sort(columns='Fly_Number', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
# OLO_mf = OLO_mf.sort(columns='Date', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
OLO_mf

,Comment,Date,Dead,Fly_Number,Sex,Species,Teneral,Trap_No,Village,infection_state
1861,head sample lost,2014-07-01,False,1,F,Gff,False,5,OLO,False
1864,whole body sample lost,2014-07-01,False,4,M,Gff,False,5,OLO,False
1865,NaN,2014-07-01,False,5,M,Gff,False,5,OLO,False
1866,NaN,2014-07-01,False,6,M,Gff,False,5,OLO,False
1867,NaN,2014-07-01,False,7,M,Gff,False,5,OLO,False
1875,NaN,2014-07-01,False,15,M,Gff,False,7,OLO,False
1876,NaN,2014-07-01,False,16,F,Gff,False,7,OLO,False
1879,NaN,2014-07-01,False,19,F,Gff,False,2,OLO,False
1882,NaN,2014-07-01,False,22,M,Gff,False,1,OLO,False
1883,NaN,2014-07-01,False,23,F,Gff,False,1,OLO,False


In [59]:
OLO = dfp.query('Dead == False').query('Sex == "M"').query('Village == "OLO"').query('Teneral == False')
OLO

,Comment,Date,Dead,Fly_Number,Hunger_stage,Kept_in,Sex,Species,Teneral,Trap_No,Village,Wing_fray,infection_state,midgut,prob,sal_gland,workbook,worksheet
1864,whole body sample lost,2014-07-01,False,4,3,EtOH,M,Gff,False,5,OLO,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Adjumani
1865,NaN,2014-07-01,False,5,2,EtOH,M,Gff,False,5,OLO,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Adjumani
1866,NaN,2014-07-01,False,6,1,EtOH,M,Gff,False,5,OLO,3,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Adjumani
1867,NaN,2014-07-01,False,7,3,EtOH,M,Gff,False,5,OLO,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Adjumani
1875,NaN,2014-07-01,False,15,3,EtOH,M,Gff,False,7,OLO,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Adjumani
1878,NaN,2014-07-01,False,18,3,EtOH,M,Gff,False,2,OLO,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Adjumani
1882,NaN,2014-07-01,False,22,3,EtOH,M,Gff,False,1,OLO,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Adjumani
1891,NaN,2014-07-01,False,31,3,EtOH,M,Gff,False,9,OLO,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Adjumani
1898,NaN,2014-08-01,False,38,3,EtOH,M,Gff,False,7,OLO,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Adjumani
1918,NaN,2014-08-01,False,58,3,EtOH,M,Gff,False,1,OLO,2,False,False,False,False,2014_spring_summer_from_rob.xlsx,Dissection Data-Adjumani


In [60]:
len(OLO)

16